<a href="https://colab.research.google.com/github/plthiyagu/Personnel/blob/master/Image_Classification_Web_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This video is supporting code for my video - https://youtu.be/AIDo3GO9t5A

In [1]:
!pip install -U ipykernel

     |████████████████████████████████| 122kB 4.9MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.3.4 which is incompatible.
  Found existing installation: ipykernel 4.10.1
    Uninstalling ipykernel-4.10.1:
      Successfully uninstalled ipykernel-4.10.1


In [1]:
!pip install -q streamlit

     |████████████████████████████████| 7.1MB 4.5MB/s 
     |████████████████████████████████| 112kB 51.3MB/s 
     |████████████████████████████████| 102kB 12.8MB/s 
     |████████████████████████████████| 4.4MB 57.9MB/s 


In [2]:
!pip install pyngrok

  Created wheel for pyngrok: filename=pyngrok-4.1.8-cp36-none-any.whl size=16337 sha256=54ef6deaefbd04988d60ee906f0ed5a5d078ff5764684dcf76286223e7b00679
  Stored in directory: /root/.cache/pip/wheels/7d/25/b7/5e4a3aefe7dd2dd9cec445786d64fe70e885109636fea356c7
Successfully built pyngrok


Update the token below from creating an account in ngrok website

You can ignore Ngrok step completely if you are not looking to expose your app to internet



In [3]:
!ngrok authtoken 1fax27GJk0gZZLd5mYbwKscD5Lh_3a3xx3tAyezcAm3yraaqN


Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
%%writefile score.py

import tensorflow as tf
import numpy as np
import streamlit as st
from PIL import Image
import requests
from io import BytesIO

st.set_option('deprecation.showfileUploaderEncoding', False)
st.title("Bean Image Classifier")
st.text("Provide URL of bean Image for image classification")

@st.cache(allow_output_mutation=True)
def load_model():
  model = tf.keras.models.load_model('/content/drive/My Drive/Colab Notebooks/models/image/models')
  return model

with st.spinner('Loading Model Into Memory....'):
  model = load_model()

classes=['angular_leaf_spot','bean_rust','healthy']

def scale(image):
  image = tf.cast(image, tf.float32)
  image /= 255.0

  return tf.image.resize(image,[224,224])

def decode_img(image):
  img = tf.image.decode_jpeg(image, channels=3)
  img = scale(img)
  return np.expand_dims(img, axis=0)

path = st.text_input('Enter Image URL to Classify.. ','http://barmac.com.au/wp-content/uploads/sites/3/2016/01/Angular-Leaf-Spot-Beans1.jpg')
if path is not None:
    content = requests.get(path).content

    st.write("Predicted Class :")
    with st.spinner('classifying.....'):
      label =np.argmax(model.predict(decode_img(content)),axis=1)
      st.write(classes[label[0]])    
    st.write("")
    image = Image.open(BytesIO(content))
    st.image(image, caption='Classifying Bean Image', use_column_width=True)    


Writing score.py


In [6]:
!nohup streamlit run score.py &

nohup: appending output to 'nohup.out'


In [7]:
from pyngrok import ngrok

url = ngrok.connect(port=8501)
url

'http://29532af7e658.ngrok.io'

In [8]:
!cat nohup.out

2020-08-04 12:13:13.975076: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.233.226.190:8501

